In [126]:
%config ZMQInteractiveShell.ast_node_interactivity='all'
%matplotlib inline
import warnings;warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import os
import matplotlib.pyplot as plt

In [162]:
# target = '标签'
target = 0
uid = '申请编号'
columns = {}
columns['no_fea'] = [uid, target, '申请时点']
columns['01'] = [
    '贷款类型',
    '是否提供手机号',
    '是否提供电话',
    '手机号是否有效',
    '是否提供email',
    '申请人是否额外提供了文件2',
    '申请人是否额外提供了文件3',
    '申请人是否额外提供了文件4',
    '申请人是否额外提供了文件5',
    '申请人是否额外提供了文件6',
    '申请人是否额外提供了文件7',
    '申请人是否额外提供了文件8',
    '申请人是否额外提供了文件9',
    '是否有车',
    '是否有房',
    '地址是否一致标志1',
    '地址是否一致标志2',
    '地址是否一致标志3',
    '地址是否一致标志4',
    '地址是否一致标志5',
    '地址是否一致标志6']
columns['time'] = [
    '申请周内日',
    '申请时点',
    '出生日期距申请日期天数']
columns['_dt'] = [
    '最近一次换手机号码距申请日天数',
    '出生日期距申请日期天数',
    '工作日期距申请日期天数',
    '注册日期距申请日期天数',
    '身份认证日期距申请日期天数']
columns['money'] = [
    '贷款年金',
    '客户收入',
    '商品价格',
    '信用额度']

In [431]:
target = '标签'
uid = '申请编号'

def calc_auc(y_test, y_proba):
    auc = round(metrics.roc_auc_score(y_test, y_proba), 3)
    return auc

def calc_ks(y_test, y_proba):
    s = pd.qcut(y_proba, 10, labels=False, duplicates='drop')
    fpr, tpr, thresholds = metrics.roc_curve(y_test, s)
    KS = round(max(abs(fpr-tpr)) * 100, 1) # 横坐标倒序
    return KS

def ks_score(y_test, y_proba):
    scale = 4
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_proba, pos_label=1)
    KS = round(max(list(tpr-fpr)), scale)
    return KS

def ks_score_rf(estimator, y_test, y_proba):
    scale = 4
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_proba, pos_label=1)
    KS = round(max(list(tpr-fpr)), scale)
    return KS

def ks_score_lgb(y_test, y_proba):
    scale = 4
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_proba, pos_label=1)
    KS = round(max(list(tpr-fpr)), scale)
    return 'KS', KS, True

In [148]:
############## Load

data_ori= pd.read_csv('./tmp/2_eda_outlier.csv', header=0, index_col=0)
data_ori.shape
data_ori.head()

# Drop uid
features_ori = data_ori.drop(uid, axis=1)
''' Ori '''
features_ori.shape

(140000, 63)

,申请编号,标签,贷款类型,信用额度,贷款年金,商品价格,陪同申请人,出生日期距申请日期天数,工作日期距申请日期天数,注册日期距申请日期天数,...,地址是否一致标志3,地址是否一致标志4,地址是否一致标志5,地址是否一致标志6,单位类型,社交圈违约信息2_2,社交圈违约信息2_1,社交圈违约信息1_2,社交圈违约信息1_1,最近一次换手机号码距申请日天数
0,0,1,0,460190.889355,17463.042019,419951.511045,7.0,-10107.0,-342.0,-5421.0,...,0,0,0,0,5,0.0,0.0,0.0,0.0,-887.0
1,1,0,0,424370.659603,15585.046388,384597.975692,7.0,-13980.0,-1110.0,-3387.0,...,0,0,0,1,5,0.0,0.0,0.0,0.0,-271.0
2,2,0,0,469330.587153,18837.558252,419951.511045,1.0,-13331.0,-2246.0,-3870.0,...,1,0,0,0,5,0.0,0.0,0.0,0.0,-332.0
3,3,0,0,464188.353140,16754.587069,409850.500944,7.0,-16540.0,NaN,-970.0,...,0,0,0,0,57,1.0,0.0,1.0,0.0,-204.0
4,5,0,0,434196.988738,16165.009200,394698.985793,7.0,-17919.0,-11037.0,-9350.0,...,1,1,0,1,53,0.0,0.0,0.0,0.0,-1049.0


' Ori '

(140000, 62)

In [212]:
############## Fill Null

from sklearn.preprocessing import Imputer

# Fill Null
imp =Imputer(missing_values=np.NaN, strategy="mean", axis=0)
# features = pd.DataFrame(imp.fit_transform(features_ori))
# features.head()
features = imp.fit_transform(features_ori)
features[0]

array([ 1.00000000e+00,  0.00000000e+00,  4.60190889e+05,  1.74630420e+04,
        4.19951511e+05,  7.00000000e+00, -1.01070000e+04, -3.42000000e+02,
       -5.42100000e+03, -3.29200000e+03,  1.00000000e+00,  1.00000000e+00,
        1.00000000e+00,  0.00000000e+00,  2.00000000e+00,  9.00000000e+00,
        4.74687967e-01,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  6.35559864e-03,  7.25051997e-03,  3.40733007e-02,
        2.89962794e-01,  2.74028223e-01,  2.27246211e+00,  1.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  2.37349581e+05,
        1.00000000e+00,  2.00000000e+00,  3.00000000e+00,  1.00000000e+00,
        2.23301980e-02,  5.84622526e+00,  6.00000000e+00,  1.00000000e+00,
        5.96305847e-01,  7.61939951e-02,  1.15704946e-01,  1.10489524e-01,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

In [164]:
############## Train, Test

from sklearn.model_selection import train_test_split

# Split
X, X_test = train_test_split(features, test_size=0.3, random_state=123)
y = X.pop(target)
y_test = X_test.pop(target)

''' Training, Test '''
X.shape
X_test.shape

' Training, Test '

(98000, 61)

(42000, 61)

In [215]:
############## k-fold

# X = data
# y = X.pop(0)
# X.shape

X = features[:, 1:]
y = features[:, 0]
X[0]
y[0]

array([ 0.00000000e+00,  4.60190889e+05,  1.74630420e+04,  4.19951511e+05,
        7.00000000e+00, -1.01070000e+04, -3.42000000e+02, -5.42100000e+03,
       -3.29200000e+03,  1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
        0.00000000e+00,  2.00000000e+00,  9.00000000e+00,  4.74687967e-01,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        6.35559864e-03,  7.25051997e-03,  3.40733007e-02,  2.89962794e-01,
        2.74028223e-01,  2.27246211e+00,  1.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  2.37349581e+05,  1.00000000e+00,
        2.00000000e+00,  3.00000000e+00,  1.00000000e+00,  2.23301980e-02,
        5.84622526e+00,  6.00000000e+00,  1.00000000e+00,  5.96305847e-01,
        7.61939951e-02,  1.15704946e-01,  1.10489524e-01,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  

1.0

In [113]:
## V1

# Governing choices for search
N_FOLDS = 5
MAX_EVALS = 5
params = {'num_boost_round' : 10000,
          'num_leaves' : 2**5,
              'feature_fraction' : 0.9,
              'bagging_fraction' : 0.8,
              'bagging_freq' : 5,
              'random_state' : 123,
              'is_unbalance' : True,
              'early_stopping_rounds' : 100}

gbm = lgb.LGBMClassifier(random_state=50)

# Data set
train = lgb.Dataset(X, y)

# CV
cv_results = lgb.cv(params, train, nfold = N_FOLDS, metrics = 'auc', verbose_eval = False, seed = 123)

# Highest score
best = cv_results['auc-mean'][-1]
best

# Standard deviation of best score
best_std = cv_results['auc-stdv'][-1]
best_std

# Optimal number of esimators found in cv
gbm.n_estimators = len(cv_results['auc-mean'])
gbm.n_estimators

In [132]:
## V2

from sklearn.model_selection import StratifiedKFold
n_splits = 2
x_score = []    # KSs
cv_pred = []    # predictions

params = {'num_boost_round' : 10000,
          'num_leaves' : 2**5,
          'feature_fraction' : 0.9,
          'bagging_fraction' : 0.8,
          'bagging_freq' : 5,
          'random_state' : 123,
          'is_unbalance' : True,
          'early_stopping_rounds' : 100}

skf = StratifiedKFold(n_splits=n_splits, random_state=123, shuffle=True)
for index, (train_index, test_index) in enumerate(skf.split(X, y)):
    train_index.shape
    # Get data
    X_train, X_valid, y_train, y_valid = X[train_index], X[test_index], y[train_index], y[test_index]
    train_data = lgb.Dataset(X_train, y_train)
    validation_data = lgb.Dataset(X_valid, y_valid)
    # Train
    clf = lgb.train(params, train_data, valid_sets=[validation_data],
                    feval=ks_score_lgb, verbose_eval=1)
    # Test
    x_pred = clf.predict(X_valid, num_iteration=clf.best_iteration)
    x_pred = [np.argmax(x) for x in x_pred]
    x_score.append(ks_score_lgb(y_valid, x_pred)  # 计算f1值
    y_test = clf.predict(X_test, num_iteration=clf.best_iteration)  # 预测
    y_test = [np.argmax(x) for x in y_test]
        

(70000,)

(70000,)

In [355]:
base_models = [LGB, RF]
base_models_ = [list() for x in base_models]
kfold = KFold(n_splits=2, shuffle=True, random_state=123)

out_of_fold_predictions = np.zeros((X.shape[0], len(base_models)))
for i, model in enumerate(base_models):
    j = 0
    for train_index, valid_index in kfold.split(X, y):
        instance = clone(model)
        base_models_[i].append(instance)
        instance.fit(X[train_index],  y[train_index])
        len(X[train_index])
#         y_pred = instance.predict(X[valid_index])
        y_pred = instance.predict_proba(X[valid_index])[:,1]
        ks = ks_score(y[valid_index], y_pred)
        print(f'* KS({j}):{ks} *')
        out_of_fold_predictions[valid_index, i] = y_pred
        j += 1

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

70000

* KS(0):0.3001 *


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

70000

...

70000

* KS(0):0.1628 *


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

70000

* KS(1):0.1485 *


In [356]:
out_of_fold_predictions[:10]

array([[0.15064948, 0.1       ],
       [0.31284796, 0.3       ],
       [0.1214973 , 0.1       ],
       [0.14398537, 0.        ],
       [0.03966171, 0.2       ],
       [0.12636723, 0.3       ],
       [0.41769283, 0.5       ],
       [0.17496637, 0.1       ],
       [0.09707957, 0.2       ],
       [0.24128142, 0.3       ]])

In [359]:
meta_model = LR
meta_model_ = clone(meta_model)

meta_model_.fit(out_of_fold_predictions, y)
y_pred = meta_model_.predict_proba(out_of_fold_predictions)[:,1]
ks_score(y, y_pred)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

0.3029

In [376]:
meta_features = np.column_stack ([
    np.column_stack(
        [model.predict_proba(X)[:,1] for model in base_models]
    ).mean (axis=1)
    for base_models in base_models_])

In [371]:
for base_models in base_models_:
    for model in base_models:
        pred = model.predict_proba(X)[:,1]
        ks_score(y, pred)

0.3737

0.3789

0.5646

0.5524

In [393]:
from sklearn.model_selection import KFold
from sklearn.base import BaseEstimator, ClassifierMixin, TransformerMixin, clone

class StackingModels (BaseEstimator, ClassifierMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds

    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=123)
        
        # Get results of basic models
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            print(f'*** Base Model: {model.__class__} ***')
            j = 0
            for train_index, valid_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index],  y[train_index])
#                 y_pred = instance.predict(X[valid_index])
                y_pred = instance.predict_proba(X[valid_index])[:,1]
                ks = ks_score(y[valid_index], y_pred)
                print(f'* KS({j}):{ks} *')
                out_of_fold_predictions[valid_index, i] = y_pred
                j += 1

#         # 将交叉验证预测出的结果 和 训练集中的标签值进行训练
#         self.meta_model_.fit(out_of_fold_predictions, y)
#         return self

    # 从得到的新的特征  采用新的模型进行预测  并输出结果
    def predict(self, X):
        meta_features = np.column_stack ([
            np.column_stack(
#                 [model.predict(X) for model in base_models]
                [model.predict_proba(X)[:,1] for model in base_models]
            ).mean(axis=1)
            for base_models in self.base_models_
        ])
#         return self.meta_model_.predict(meta_features)
        return meta_features.mean(axis=1)

In [394]:
RF = joblib.load('./model/base_rf.m')

CLF = StackingModels(base_models=[LGB, XGB, RF], meta_model=LR)
CLF.fit(X, y)
pred = CLF.predict(X)
ks_score(y, pred)

*** Base Model: <class 'lightgbm.sklearn.LGBMClassifier'> ***
* KS(0):0.3204 *
* KS(1):0.3107 *
* KS(2):0.2957 *
* KS(3):0.312 *
* KS(4):0.3029 *
*** Base Model: <class 'xgboost.sklearn.XGBClassifier'> ***
* KS(0):0.2987 *
* KS(1):0.2928 *
* KS(2):0.2856 *
* KS(3):0.2955 *
* KS(4):0.2937 *
*** Base Model: <class 'sklearn.ensemble.forest.RandomForestClassifier'> ***
* KS(0):0.169 *
* KS(1):0.1777 *
* KS(2):0.1729 *
* KS(3):0.1536 *
* KS(4):0.1576 *


0.8961

In [455]:
########## Grid Search

from sklearn.model_selection import RandomizedSearchCV
from sklearn.externals import joblib

param_fixed_search = {
    'n_iter' : 2,
    'cv' : 2, 
    'scoring' : 'roc_auc', 
    'n_jobs' : -1, 
    'random_state' : 123, 
    'verbose' : 1}

In [456]:
from sklearn.ensemble import RandomForestClassifier

param_dist = {
#     'n_estimators' : range(100,500,50),
    'n_estimators' : range(50,100,10),
    'max_features' : ['sqrt', 'log2'],
    'max_depth' : range(2, 10),
    'min_samples_split' : range(50, 100, 10),
    'min_samples_leaf' : range(50, 100, 10),
    'criterion' : ['gini', 'entropy']}
param_fixed_model = {
    'oob_score' : True,
    'random_state':123,
    'verbose':0}

''' Baseline '''
rf = RandomForestClassifier(**param_fixed_model)
rf.fit(X, y)
pred = rf.predict_proba(X)
ks_score(y, pred[:,1])

''' Best '''
grid = RandomizedSearchCV(RandomForestClassifier(**param_fixed_model), param_dist, **param_fixed_search)
grid.fit(X, y)
grid.best_score_
best_params = grid.best_params_
# Dump
RF = RandomForestClassifier(**best_params)
joblib.dump(RF, './model/base_rf.m')

''' Test '''
rf = clone(RF)
rf.fit(X, y)
pred = rf.predict_proba(X)
ks_score(y, pred[:,1])

' Baseline '

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=123, verbose=0, warm_start=False)

0.9842

' Best '

Fitting 2 folds for each of 2 candidates, totalling 4 fits


...

0.685744518607443

['./model/base_rf.m']

' Test '

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=9, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=90, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=80, n_jobs=None,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

0.314

In [457]:
rf1 = joblib.load('./model/base_rf.m')
# rf.fit(X, y)
pred = rf1.predict_proba(X)
ks_score(y, pred[:,1])

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [313]:
import xgboost as xgb

param_dist = {
    'n_estimators' : range(50,100,10),
    'subsample' : np.linspace(0.7,0.9,2)
    'max_depth':range(2,15,1),
    'eta' : np.linspace(0.1,0.5,2)
    'learning_rate':np.linspace(0.01,2,20),
    
    'colsample_bytree':np.linspace(0.5,0.98,10),
    'min_child_weight':range(1,9,1)}
XGB = xgb.XGBClassifier()



param_dist = {
    'max_depth' : range(3,10),
    'min_samples_leaf' : range(50, 100, 10),
    
    'criterion' : ['gini', 'entropy'],
    'oob_score' : [True, False]}
param_fixed = {
    'random_state':123,
    'verbose':0}

''' Baseline '''
rf = RandomForestClassifier(**param_fixed)
rf.fit(X, y)
pred = rf.predict_proba(X)
ks_score(y, pred[:,1])

''' Best '''
grid = RandomizedSearchCV(RandomForestClassifier(**param_fixed), param_dist, n_iter=2, cv=2, 
                          scoring='roc_auc', n_jobs=-1, random_state=123, verbose=1)
grid.fit(X, y)
grid.best_score_
best_params = grid.best_params_
# Dump
RF = RandomForestClassifier(**best_params)
joblib.dump(RF, './model/base_rf.m')

''' Test '''
rf = clone(RF)
rf.fit(X, y)
pred = rf.predict_proba(X)
ks_score(y, pred[:,1])

In [458]:
from hyperopt import fmin, tpe, hp,space_eval,rand,Trials,partial,STATUS_OK

# 定义一个目标函数,接受一个变量,计算后返回一个函数的损失值，
def GBM(argsDict):
    max_depth = argsDict["max_depth"] + 5
    n_estimators = argsDict['n_estimators'] * 5 + 50
    learning_rate = argsDict["learning_rate"] * 0.02 + 0.05
    subsample = argsDict["subsample"] * 0.1 + 0.7
    min_child_weight = argsDict["min_child_weight"]+1
    global attr_train,label_train

    gbm = xgb.XGBClassifier(nthread=4,    #进程数
                            max_depth=max_depth,  #最大深度
                            n_estimators=n_estimators,   #树的数量
                            learning_rate=learning_rate, #学习率
                            subsample=subsample,      #采样数
                            min_child_weight=min_child_weight,   #孩子数
                            max_delta_step = 10,  #10步不降则停止
                            objective="binary:logistic")

    metric = cross_val_score(gbm,attr_train,label_train,cv=5,scoring="roc_auc").mean()
    print(metric)
    return metric

# 定义参数的搜索空间

space = {"max_depth":hp.randint("max_depth",15),
         "n_estimators":hp.randint("n_estimators",10),  #[0,1,2,3,4,5] -> [50,]
         "learning_rate":hp.randint("learning_rate",6),  #[0,1,2,3,4,5] -> 0.05,0.06
         "subsample":hp.randint("subsample",4),#[0,1,2,3] -> [0.7,0.8,0.9,1.0]
         "min_child_weight":hp.randint("min_child_weight",5), #
        }
algo = partial(tpe.suggest, n_startup_jobs=1)  # 定义随机搜索算法。搜索算法本身也有内置的参数决定如何去优化目标函数
best = fmin(GBM, space, algo=algo, max_evals=4)  # 对定义的参数范围，调用搜索算法，对模型进行搜索

print(best)
print(GBM(best))

ModuleNotFoundError: No module named 'hyperopt'

In [314]:
import lightgbm as lgb

# params_lgb = {'num_boost_round' : 10000,
#           'num_leaves' : 2**5,
#           'feature_fraction' : 0.9,
#           'bagging_fraction' : 0.8,
#           'bagging_freq' : 5,
#           'random_state' : 123,
#           'is_unbalance' : True,
#           'early_stopping_rounds' : 100}
LGB = lgb.LGBMClassifier()

# LGB.fit(X, y)
# pred = LGB.predict_proba(X_test)
# ks_score(y_test, pred[:,1])

In [315]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()

# LR.fit(X, y)
# pred = LR.predict_proba(X_test)
# ks_score(y_test, pred[:,1])

In [102]:
# ### Sample Balance
#
# data_ori = data_ori.drop(uid, axis=1)
# ''' Ori '''
# data_ori.shape
# 
# data1 = data.loc[data[target]==1, :]
# data0 = data.loc[data[target]==0, :]
# frac = len(data1) / len(data0)
# data0 = data0.sample(frac=frac, random_state=123)
# data = pd.concat([data1, data0], axis=0)
# ''' Balance '''
# data[target].value_counts()

# # Fill Null
# data = data[data.columns.difference(columns['_dt'] + columns['money'])]
# data.fillna(-1, inplace=True)

In [111]:
############### LGB Training

def train_lgb(X, y):
    # Data input
    train = lgb.Dataset(X, y)
    params = {'num_leaves' : 2**5,
              'feature_fraction' : 0.9,
              'bagging_fraction' : 0.8,
              'bagging_freq' : 5,
              'random_state' : 123,
              'is_unbalance' : True,
              'early_stopping_rounds' : 100
              }
    clf = lgb.train(params,
                    train)
    pred = clf.predict(X)
    ks = ks_score(y, pred)
    print(f'Training KS: {ks}')
    return clf

def test_lgb(clf, X):
    pred = clf.predict(X, num_iterations=clf.best_iteration)
    return pred


### Training 
print('************************ Training ************************')

gbm = train_lgb(X, y)
gbm.save_model(f'./model/1_model_lgb.txt')

print('************************ Test ************************')

# print('* Training *')
# train['score_lgb'] = test_lgb(gbm, X)
# auc = calc_auc(train, 'score_lgb', target)
# ks = calc_ks(train, 'score_lgb', target)
# print(f'AUC:{auc}\tKS:{ks}')

X_test['score_lgb'] = test_lgb(gbm, X_test)
ks = ks_score(y_test, X_test['score_lgb'])
print(f'Test KS: {ks}')

************************ Training ************************
Training KS: 0.3946


************************ Test ************************
Test KS: 0.3209


In [401]:
############### Test

def set_outlier(col):
    if col < col_min:
        col = col_min
    elif col > col_max:
        col = col_max
    return col

###################### Load 

''' Ori '''
d1 = pd.read_csv('./data/A_Application.csv')
d1.shape
d2 = pd.read_csv('./data/A_Personas.csv')
d2.shape
d11 = pd.merge(d1, d2, how='inner', on='申请编号')
d11.shape

###################### Data Prepare 

X_test = d11.copy()
uids = X_test.pop(uid)

### 置空
X_test.loc[X_test['孩子个数']==-1, '孩子个数'] = np.NaN
# 距申请日期>0
for col in columns['_dt']:
    X_test.loc[X_test[col]>0, col] = np.NaN
    
### Outlier
outliers = pd.read_csv('./tmp/0_outlier1.csv', index_col=0)

for col in outliers.index:
    col_min = outliers.loc[col, 'min']
    col_max = outliers.loc[col, 'max']
    X_test[col] = X_test[col].apply(set_outlier)
    
# Fill Null
imp =Imputer(missing_values=np.NaN, strategy="mean", axis=0)
X_test = imp.fit_transform(X_test)

''' Test '''
len(X_test)
len(X_test[0])

' Ori '

(21511, 31)

(21511, 32)

(21511, 62)

(61, 2)

' Test '

21511

61

In [417]:
def norm_score(score):
    if score < 0:
        score = 0
    elif score > 1:
        score = 1
    return score

###################### Predict

scores = CLF.predict(X_test)
out = pd.concat([uids, pd.Series(scores)], axis=1)
out[0] = out[0].apply(norm_score)
out.head()

out.to_csv('./model/predict.csv', header=False, index=False)

,申请编号,0
0,122687,0.082994
1,32425,0.102863
2,2024,0.096765
3,25019,0.175065
4,162532,0.187137
